In [3]:
import numpy as np

#csr from scartch
#value -> stores all non zero values in row major order
#col_indices -> stores the column index for each non-zero value
#row_ptr -> stores the index ranges for each row in the other arrays

class CSR_matrix:
    def __init__(self, dense_matrix):
        self.rows= len(dense_matrix)
        self.cols = len(dense_matrix[0]) if self.rows>0 else 0
        self.values= []
        self.col_indices = []
        self.row_ptr = [0]

        for row in dense_matrix:
            #number of zero in row
            nnz_in_row = 0
            for col_idx, value in enumerate(row):
                if value!=0:
                    self.values.append(value)
                    self.col_indices.append(col_idx)
                    nnz_in_row+=1
            self.row_ptr.append(self.row_ptr[-1]+nnz_in_row)

    def get_rows(self, row_idx):
        start = self.row_ptr[row_idx]
        end = self.row_ptr[row_idx+1]
        row = [0]*self.cols
        for i in range(start, end):
            col = self.col_indices[i]
            row[col] = self.values[i]
        return row
    
    def to_dense(self):
        
        dense =[[0]*self.cols for _ in range(self.rows)]
        #go through the rows
        for row_idx in range(self.rows):
            start = self.row_ptr[row_idx]
            end =  self.row_ptr[row_idx+1]
            #for each row, we count how many nonzero rows
            #then we go through these nonzero elements and put them back
            for i in range(start, end):
                col_idx = self.col_indices[i]
                dense[row_idx][col_idx] = self.values[i]

        return (dense)
    
    def matvec(self, vector):
        result = np.zeros(self.rows)
        for i in range(self.rows):
            start = self.row_ptr[i]
            end = self.row_ptr[i+1]
            row_sum = 0
            for j in range(start, end):
                row_sum+=self.values[j]*vector[self.col_indices[j]]
            result[i] = row_sum

        return result
    
    



dense_matrix=[[1, 0, 2],
 [0, 0, 3],
 [4, 5, 0]]

csr = CSR_matrix(dense_matrix)


In [4]:
#test 1 

# Test 1: Basic 3x3 matrix
dense_matrix = [
    [1, 0, 2],
    [0, 0, 3],
    [4, 5, 0]
]

csr = CSR_matrix(dense_matrix)

# Test to_dense()
print("Original matrix:")
print(np.array(dense_matrix))
print("\nReconstructed from CSR:")
print(np.array(csr.to_dense()))

# Test matvec()
vector = [7, 8, 9]
print("\nMatrix-vector multiplication (A*x):")
print(f"Expected: [25, 27, 68]")  # 1*7 + 2*9 = 25; 3*9 = 27; 4*7 + 5*8 = 68
print(f"Actual: {csr.matvec(vector)}")

# Test get_rows()
print("\nGetting individual rows:")
print(f"Row 0: {csr.get_rows(0)} (expected [1, 0, 2])")
print(f"Row 1: {csr.get_rows(1)} (expected [0, 0, 3])")
print(f"Row 2: {csr.get_rows(2)} (expected [4, 5, 0])")

Original matrix:
[[1 0 2]
 [0 0 3]
 [4 5 0]]

Reconstructed from CSR:
[[1 0 2]
 [0 0 3]
 [4 5 0]]

Matrix-vector multiplication (A*x):
Expected: [25, 27, 68]
Actual: [25. 27. 68.]

Getting individual rows:
Row 0: [1, 0, 2] (expected [1, 0, 2])
Row 1: [0, 0, 3] (expected [0, 0, 3])
Row 2: [4, 5, 0] (expected [4, 5, 0])


In [5]:
#test 2
# Test 2: Empty matrix
empty_matrix = []
csr_empty = CSR_matrix(empty_matrix)
print("\nEmpty matrix test:")
print(f"to_dense(): {csr_empty.to_dense()} (expected [])")
print(f"matvec([]): {csr_empty.matvec([])} (expected [])")


Empty matrix test:
to_dense(): [] (expected [])
matvec([]): [] (expected [])


In [6]:
#test 3
# Test 3: All zeros matrix
zero_matrix = [
    [0, 0, 0],
    [0, 0, 0]
]
csr_zero = CSR_matrix(zero_matrix)
vector = [1, 2, 3]

print("\nAll zeros matrix test:")
print("Reconstructed from CSR:")
print(np.array(csr_zero.to_dense()))
print(f"matvec: {csr_zero.matvec(vector)} (expected [0, 0])")


All zeros matrix test:
Reconstructed from CSR:
[[0 0 0]
 [0 0 0]]
matvec: [0. 0.] (expected [0, 0])


In [7]:
#test 4
# Test 4: Diagonal matrix
diag_matrix = [
    [5, 0, 0],
    [0, 3, 0],
    [0, 0, 1]
]
csr_diag = CSR_matrix(diag_matrix)
vector = [2, 4, 6]

print("\nDiagonal matrix test:")
print("Reconstructed from CSR:")
print(np.array(csr_diag.to_dense()))
print(f"matvec: {csr_diag.matvec(vector)} (expected [10, 12, 6])")


Diagonal matrix test:
Reconstructed from CSR:
[[5 0 0]
 [0 3 0]
 [0 0 1]]
matvec: [10. 12.  6.] (expected [10, 12, 6])


In [8]:
#test 5
# Test 5: Full matrix (no zeros)
full_matrix = [
    [1, 2, 3],
    [4, 5, 6]
]
csr_full = CSR_matrix(full_matrix)
vector = [1, 0, -1]

print("\nFull matrix test:")
print("Reconstructed from CSR:")
print(np.array(csr_full.to_dense()))
print(f"matvec: {csr_full.matvec(vector)} (expected [-2, -2])")  # 1*1 + 2*0 + 3*-1 = -2; 4*1 + 5*0 + 6*-1 = -2


Full matrix test:
Reconstructed from CSR:
[[1 2 3]
 [4 5 6]]
matvec: [-2. -2.] (expected [-2, -2])


In [9]:
#test 6
# Test 6: Single element matrix
single_matrix = [[7]]
csr_single = CSR_matrix(single_matrix)
vector = [3]

print("\nSingle element matrix test:")
print("Reconstructed from CSR:")
print(np.array(csr_single.to_dense()))
print(f"matvec: {csr_single.matvec(vector)} (expected [21])")


Single element matrix test:
Reconstructed from CSR:
[[7]]
matvec: [21.] (expected [21])


In [10]:
#test 7
# Test 7: Rectangular matrix
rect_matrix = [
    [1, 0, 2, 0],
    [0, 3, 0, 4],
    [5, 0, 6, 0]
]
csr_rect = CSR_matrix(rect_matrix)
vector = [1, 2, 3, 4]

print("\nRectangular matrix test:")
print("Reconstructed from CSR:")
print(np.array(csr_rect.to_dense()))
print(f"matvec: {csr_rect.matvec(vector)} (expected [7, 22, 23])")  # 1*1 + 2*3 = 7; 3*2 + 4*4 = 22; 5*1 + 6*3 = 23


Rectangular matrix test:
Reconstructed from CSR:
[[1 0 2 0]
 [0 3 0 4]
 [5 0 6 0]]
matvec: [ 7. 22. 23.] (expected [7, 22, 23])


In [13]:
# Test 8: Random large matrix
np.random.seed(42)
large_dense = np.random.rand(1000, 1000)
large_dense[large_dense < 0.9] = 0  # Make it sparse (~10% non-zero)
csr_large = CSR_matrix(large_dense.tolist())

# Verify reconstruction
reconstructed = np.array(csr_large.to_dense())
print("\nLarge matrix test:")
print(f"Original and reconstructed equal? {np.allclose(large_dense, reconstructed)}")

# Test matvec with random vector
large_vector = np.random.rand(1000)
result = csr_large.matvec(large_vector)
expected = large_dense @ large_vector
print(f"matvec correct? {np.allclose(result, expected)}")


Large matrix test:
Original and reconstructed equal? True
matvec correct? True
